In [147]:
import numpy as np
import cvxpy as cp

**1**

Хотим минимизировать $c^T x$ при условии $Ax = b, x \ge 0$.

$L(x, \lambda, \mu) = c^T x - \lambda^T x + \mu^T (Ax - b)$

$q(\lambda, \mu) = \begin{cases} -\mu^T b,\ c^T - \lambda^T + \mu^T A = 0\\ -\infty,\ c^T - \lambda^T + \mu^T A \neq 0 \end{cases} $

Двойственная задача: хотим максимизировать $-\mu^T b$ при условии $c^T - \lambda^T + \mu^T A = 0, \lambda \ge 0$.

In [148]:
def solve_dual_task(A, b, c):
    mu, lambda_ = cp.Variable(A.shape[0]), cp.Variable(A.shape[1])
    cp.Problem(
        cp.Minimize(mu.T @ b),
        [c.T - lambda_.T + mu.T @ A == 0, lambda_ >= 0]
    ).solve()
    return mu.value, lambda_.value

In [149]:
A = np.array([[3, 2], [3, 4]]) 
b = np.array([4, 6])
c = np.array([41, 13])

In [150]:
solve_dual_task(A, b, c)

(array([-20.83333333,   7.16666667]),
 array([ 1.98992768e-10, -9.81393086e-11]))

Заметим, что для ненулевых $\lambda_i$ соответствующий $x_i = 0$, значит, достаточно решить систему $Ax = b$ для остальных.

In [151]:
def solve(A, b, c):
    _, lambda_ = solve_dual_task(A, b, c)
    x = np.zeros(A.shape[1])
    zero_lambda = np.isclose(lambda_, 0)
    A_ = A[:, zero_lambda].T
    x[zero_lambda] = np.linalg.inv(A_ @ A_.T) @ A_ @ b
    return x

In [152]:
solve(A, b, c)

array([0.66666667, 1.        ])

**2**

Хотим минимизировть максимальное из расстояний от центра до точек.

In [199]:
def min_ball(X):
    c = cp.Variable((1, X.shape[1]))
    r = cp.Problem(
        cp.Minimize(
            cp.max(cp.sum((c - X) ** 2, 1))
        )
    )
    r.solve()
    return c.value, np.sqrt(r.value)

In [200]:
min_ball(np.array([
    [0, 3, 4],
    [5, 0, 0],
    [-5, 0, 0],
    [-1, 0, -2],
    [1, 0, 2]
]))

(array([[2.97910181e-10, 1.17539003e-04, 2.13752644e-04]]), 5.000000006248471)

**3**

Хотим максимизироваать $\min \limits_{y_i = 1} a^T x_i - \max \limits_{y_i = 0} a^T x_i$, что то же самое что минимизировать $p - q$ при условии $p \ge \max \limits_{y_i = 0} a^T x_i, q \le \min \limits_{y_i = 1} a^T x_i$. Будем считать что вектор $a$ лежит в единичном шаре.

In [226]:
def SVM(X, Y):
    a = cp.Variable((1, X.shape[1]))
    p, q = cp.Variable(2)
    cp.Problem(cp.Minimize(p - q),
               [p >= a @ x for x in X[Y == 0]] + \
               [q <= a @ x for x in X[Y == 1]] + \
               [cp.norm(a) <= 1]).solve()
    return a.value

In [227]:
SVM(np.array([[1, 2, 4], [3, 5, 6], [7, 8, -3]]), np.array([0, 1, 1]))

array([[0.49436073, 0.73467497, 0.46460753]])